In [1]:
import torch
import torch.nn as nn
from torch.distributions import constraints

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO, TraceTMC_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.ops.indexing import Vindex
from pyro.optim import Adam

In [2]:
data = poly.load_data(poly.JSB_CHORALES)

In [3]:
seqs = data['train']['sequences']
lengths = data['train']['sequence_lengths']

In [4]:
seqs.shape

torch.Size([229, 129, 88])

In [5]:
num_seqs, max_len, data_dim = map(int, seqs.shape)
assert lengths.shape == (num_seqs,)
assert lengths.max() == max_len

In [6]:
print(num_seqs, max_len, data_dim)

229 129 88


In [7]:
hd = 4

In [8]:
probs_x = pyro.sample("probs_x", dist.Dirichlet(0.9 * torch.eye(hd) + 0.1).to_event(1))

In [9]:
probs_x.shape

torch.Size([4, 4])

In [10]:
probs_y = pyro.sample(
            "probs_y",
            dist.Beta(0.1, 0.9).expand([hd, 2, data_dim]).to_event(3),
        )

In [11]:
probs_y.shape

torch.Size([4, 2, 88])

In [12]:
batch_size = 10

In [13]:
tones_plate = pyro.plate("tones", data_dim, dim=-1)

In [14]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'sys', 'os', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'torch', 'nn', 'constraints', 'pyro', 'poly', 'dist', 'poutine', 'SVI', 'JitTraceEnum_ELBO', 'TraceEnum_ELBO', 'TraceTMC_ELBO', 'AutoDelta', 'Vindex', 'Adam', '_i2', 'data', '_i3', 'seqs', 'lengths', '_i4', '_4', '_i5', 'num_seqs', 'max_len', 'data_dim', '_i6', '_i7', 'hd', '_i8', 'probs_x', '_i9', '_9', '_i10', 'probs_y', '_i11', '_11', '_i12', 'batch_size', '_i13', 'tones_plate', '_i14'])

In [15]:
with pyro.plate("sequences", num_seqs, batch_size, dim=-2) as batch:
    print(batch)
    lengths = lengths[batch]
    x, y = 0, 0
    print(lengths)
    for t in pyro.markov(range(lengths.max())):
        print(t)
        with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
            x = pyro.sample(
                    "x_{}".format(t),
                    dist.Categorical(probs_x[x]),
                    infer={"enumerate": "parallel"},
                )
            print(x)

tensor([ 94, 194,  57, 223, 219, 128,  54, 225,  34,  69])
tensor([41, 53, 65, 49, 65, 33, 65, 51, 48, 41])
0
tensor([[2],
        [0],
        [2],
        [2],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])
1
tensor([[2],
        [0],
        [2],
        [2],
        [0],
        [2],
        [0],
        [0],
        [0],
        [0]])
2
tensor([[2],
        [2],
        [2],
        [2],
        [0],
        [2],
        [0],
        [0],
        [2],
        [2]])
3
tensor([[2],
        [2],
        [2],
        [2],
        [0],
        [2],
        [0],
        [2],
        [2],
        [2]])
4
tensor([[2],
        [2],
        [2],
        [2],
        [0],
        [2],
        [0],
        [2],
        [2],
        [2]])
5
tensor([[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [0],
        [2],
        [2],
        [2]])
6
tensor([[2],
        [2],
        [2],
        [2],
        [2],
        [2],
        [0],
 